**YOLOV8 IMPORT**

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import ultralytics

from ultralytics import YOLO
model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 23.0MB/s]


**DEEPSORT IMPORT**

In [3]:
!git clone https://github.com/granthikhalder/deep_sort

Cloning into 'yolov8_deepsort'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 84 (delta 4), reused 45 (delta 4), pack-reused 36
Receiving objects: 100% (84/84), 51.25 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [5]:
from deep_sort.deep_sort import DeepSort

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
deep_sort = DeepSort(model_path=deep_sort_weights)

**OBJECT CLASS NAMES**

In [6]:
class_names = model.names

**YOLOV8 & DEEPSORT IMPLEMENTATION**

In [10]:
import cv2
import numpy as np
from tqdm import tqdm
import torch

input_video_path = 'input.mp4'
output_video_path = 'output_yolo_deepsort.mp4'
cap = cv2.VideoCapture(input_video_path)

# Video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Progress bar
pbar = tqdm(total=total_frames, desc='Processing video')

track_id_to_class_id = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO
    results = model(frame)

    detections = results[0].boxes.data.cpu().numpy()

    bbox_xywh = []
    confidences = []
    class_ids = []

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection[:6]
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1
        bbox_xywh.append([x_center, y_center, width, height])
        confidences.append(conf)
        class_ids.append(int(cls))

    bbox_xywh = np.array(bbox_xywh)
    confidences = np.array(confidences)
    class_ids = np.array(class_ids)

    # DEEPSORT Tracker
    outputs = deep_sort.update(bbox_xywh, confidences, frame)

    # Draw the tracking results
    for output in outputs:
        x1, y1, x2, y2, track_id = output[:5]

        # Map the track ID to the class ID
        if track_id not in track_id_to_class_id:
            track_id_to_class_id[track_id] = class_ids[0]  # assign the first class ID for this example

        class_id = track_id_to_class_id[track_id]
        class_name = class_names[class_id]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Output video
    out.write(frame)

    # Progress bar Update
    pbar.update(1)

cap.release()
out.release()
cv2.destroyAllWindows()
pbar.close()


Processing video:   0%|          | 0/350 [00:00<?, ?it/s]

Processing video:   7%|▋         | 25/350 [00:14<03:05,  1.75it/s]

0: 384x640 2 persons, 163.4ms
Speed: 6.5ms preprocess, 163.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)




Processing video:   0%|          | 1/350 [00:00<01:57,  2.98it/s]


0: 384x640 3 persons, 148.0ms
Speed: 3.7ms preprocess, 148.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


...


Processing video:  99%|█████████▉| 348/350 [02:26<00:00,  2.28it/s]


0: 384x640 1 person, 236.9ms
Speed: 9.3ms preprocess, 236.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)



Processing video: 100%|█████████▉| 349/350 [02:27<00:00,  2.42it/s]


0: 384x640 1 person, 177.5ms
Speed: 3.8ms preprocess, 177.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)



Processing video: 100%|██████████| 350/350 [02:27<00:00,  2.37it/s]


**YOLOV8, DEEPSORT, SEMANTIC SEGMENTATION IMPLEMENTATION**

In [16]:
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision.models.segmentation.deeplabv3 import DeepLabV3_ResNet101_Weights

# Initialize video capture
input_video_path = 'input.mp4'
output_video_path = 'output_yolo_deepsort_semantic.mp4'
cap = cv2.VideoCapture(input_video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize the progress bar
pbar = tqdm(total=total_frames, desc='Processing video')

# Load DeepLab model
segmentation_model = deeplabv3_resnet101(weights=DeepLabV3_ResNet101_Weights.DEFAULT)
segmentation_model.eval()

# Define transformations for DeepLab
transform_dl = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((height, width)),
    transforms.ToTensor(),
])

track_id_to_class_id = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Semantic segmentation to get masks
    input_tensor = transform_dl(frame).unsqueeze(0)
    with torch.no_grad():
        output = segmentation_model(input_tensor)['out'][0]
    masks = torch.argmax(output, dim=0).cpu().numpy()

    # YOLO to detect objects
    results = model(frame)

    # Extract the detection results
    detections = results[0].boxes.data.cpu().numpy()  # assuming results is a list with one element containing the detections

    bbox_xywh = []
    confidences = []
    class_ids = []

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection[:6]
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1
        bbox_xywh.append([x_center, y_center, width, height])
        confidences.append(conf)
        class_ids.append(int(cls))

    # Convert to numpy arrays
    bbox_xywh = np.array(bbox_xywh)
    confidences = np.array(confidences)
    class_ids = np.array(class_ids)

    # Update the DeepSORT tracker
    outputs = deep_sort.update(bbox_xywh, confidences, frame)

    # Draw the tracking results on the frame
    for output in outputs:
        x1, y1, x2, y2, track_id = output[:5]

        # Map the track ID to the class ID
        if track_id not in track_id_to_class_id:
            track_id_to_class_id[track_id] = class_ids[0]  # assign the first class ID for this example

        class_id = track_id_to_class_id[track_id]
        class_name = class_names[class_id]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Semantic segmentation masks to the frame
    for i in range(1, masks.max() + 1):  # Start from 1 to skip background
        mask = (masks == i).astype(np.uint8) * 255
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame, contours, -1, (0, 0, 255), 2)  # Draw contours of each mask

    # Write Output video
    out.write(frame)

    # Progress bar
    pbar.update(1)

cap.release()
out.release()
cv2.destroyAllWindows()
pbar.close()

Processing video:   0%|          | 0/78 [01:18<?, ?it/s]



0: 384x640 2 persons, 203.2ms
Speed: 7.6ms preprocess, 203.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


Processing video:   1%|▏         | 1/78 [00:17<22:14, 17.33s/it]


0: 384x640 4 persons, 157.0ms
Speed: 3.6ms preprocess, 157.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


Processing video:   3%|▎         | 2/78 [00:33<20:44, 16.37s/it]

...

Processing video:  97%|█████████▋| 76/78 [20:28<00:32, 16.12s/it]


0: 384x640 2 persons, 191.2ms
Speed: 3.9ms preprocess, 191.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing video:  99%|█████████▊| 77/78 [20:44<00:16, 16.16s/it]


0: 384x640 2 persons, 1 skateboard, 179.6ms
Speed: 7.5ms preprocess, 179.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)


Processing video: 100%|██████████| 78/78 [21:00<00:00, 16.16s/it]
